<a href="https://colab.research.google.com/github/manjotmb20/EC3/blob/master/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from numpy import linalg as LA


**Preprocessing of the titanic dataset**

In [0]:
df_train = pd.read_csv('train.csv')
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)
df_train[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)
df_train[['SibSp', 'Survived']].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)
df_train[['Parch', 'Survived']].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)
df_train = df_train.drop(['Ticket', 'Cabin'], axis=1)
combine = [df_train]
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract('([A-Za-z]+)\.', expand=False)
pd.crosstab(df_train['Title'], df_train['Sex'])
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Don', 'Sir', 'Jonkheer', 'Dona'],'Royalty')
    dataset['Title'] = dataset['Title'].replace(['Capt', 'Col','Dr','Major','Rev'],'Special')
df_train[['Title','Survived']].groupby(['Title'], as_index=False).mean()
title_mapping = {"Master": 1, "Miss": 2, "Mrs": 3, "Mr": 4, "Royalty": 5, "Special": 6}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
df_train = df_train.drop(['Name', 'PassengerId'], axis=1)
combine = [df_train]
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
median_age = np.zeros((2,3))
for dataset in combine:
    for sex in range(0,2):
        for pclass in range(0,3):
            guess_df = dataset[(dataset['Sex'] == sex) & \
                                  (dataset['Pclass'] == pclass+1)]['Age'].dropna()
            age_guess = guess_df.median()
            median_age[sex,pclass] = age_guess

for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = median_age[i,j]

dataset['Age'] = dataset['Age'].astype(int)
df_train['AgeBand'] = pd.cut(df_train['Age'], 5)
df_train[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)
for dataset in combine:
    dataset.loc[ dataset['Age'] <= 16, 'AgeG'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'AgeG'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'AgeG'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'AgeG'] = 3
    dataset.loc[ dataset['Age'] > 64, 'AgeG'] = 4
df_train = df_train.drop(['AgeBand','Age'], axis=1)
combine = [df_train]
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

df_train[['FamilySize','Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False) 
df_train = df_train.drop(['SibSp', 'Parch'], axis=1)
combine = [df_train]
freq_port = df_train.Embarked.dropna().mode()[0]
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
df_train[['Embarked','Survived']].groupby(['Embarked'], as_index = False).mean().sort_values(by='Survived', ascending=False)
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
df_train = df_train.drop(['Fare'], axis=1)
combine = [df_train]
for dataset in combine:
    dataset.AgeG = dataset.AgeG.astype(int)
X_train = df_train.drop("Survived", axis=1)
Y_train = df_train['Survived']           

In [13]:
X_train.shape

(891, 6)

**We create different models, we take in to account the ensemble of Support vector classifiers and Logistic Regression. **

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
classifier3=SVC(probability=True)
from sklearn.metrics import roc_auc_score,accuracy_score
x_train,x_val,y_train,y_val=train_test_split(X_train,Y_train,test_size=0.2,random_state=9)
classifier=LogisticRegression(random_state=0)
classifier.fit(x_train,y_train)
ypred=classifier.predict(x_val)
print("Accuracy score of LR",roc_auc_score(y_val,ypred))
y_prob=classifier.predict_proba(x_val)
df=pd.DataFrame()
df['pred']=ypred
survived=pd.get_dummies(df.pred,prefix='Survived')
df=pd.concat([df,survived],axis=1)
df.drop('pred',inplace=True,axis=1)
df2=pd.DataFrame()
classifier3.fit(x_train,y_train)
ypred3=classifier3.predict(x_val)
print("Accuracy score of SVM",roc_auc_score(y_val,ypred3))
df2['pred']=ypred3
y_prob3=classifier3.predict_proba(x_val)
survived2=pd.get_dummies(df2.pred,prefix='Surviveds')
df=pd.concat([df,survived2],axis=1)
'''from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2,init='k-means++')
kmeans.fit(x_train,y_train)
k_pred=kmeans.predict(x_val)
accuracy_score(y_val,k_pred)
df_kmeans=pd.DataFrame()
df_kmeans['Survived_k']=k_pred
survivedk=pd.get_dummies(df_kmeans.Survived_k,prefix='Survivedk')
df['Survivedk_0']=survivedk['Survivedk_0']
df['Survivedk_1']=survivedk['Survivedk_1']
print("Accuracy score of Kmeans",accuracy_score(y_val,k_pred))'''

Accuracy score of LR 0.7487974683544305
Accuracy score of SVM 0.7654430379746835


'from sklearn.cluster import KMeans\nkmeans = KMeans(n_clusters=2,init=\'k-means++\')\nkmeans.fit(x_train,y_train)\nk_pred=kmeans.predict(x_val)\naccuracy_score(y_val,k_pred)\ndf_kmeans=pd.DataFrame()\ndf_kmeans[\'Survived_k\']=k_pred\nsurvivedk=pd.get_dummies(df_kmeans.Survived_k,prefix=\'Survivedk\')\ndf[\'Survivedk_0\']=survivedk[\'Survivedk_0\']\ndf[\'Survivedk_1\']=survivedk[\'Survivedk_1\']\nprint("Accuracy score of Kmeans",accuracy_score(y_val,k_pred))'

In [16]:
x_val.shape

(179, 6)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=6, units=3, kernel_initializer="uniform")`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=2, kernel_initializer="uniform")`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  if __name__ == '__main__':


In [31]:
x_train.shape

(712, 6)

In [46]:
import keras
from keras.models import Sequential
from keras.layers import Conv1D,MaxPooling1D,Dense,Flatten
classifier = Sequential()
classifier.add(Dense(output_dim = 3, init = 'uniform', activation = 'relu', input_dim = 6))
#Hidden layer
classifier.add(Dense(output_dim = 2, init = 'uniform', activation = 'relu'))
#output layer with 1 output neuron which will predict 1 or 0
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train, y_train, epochs=1000, verbose=0)
prediction = classifier.predict(x_val).tolist()
# list to series
se = pd.Series(prediction)
# creating new column of predictions in data_check dataframe
data_check=pd.Series()
data_check['check'] = se
data_check['check'] = data_check['check'].str.get(0)
series = []
for val in data_check.check:
    if val >= 0.5:
        series.append(1)
    else:
        series.append(0)
roc_auc_score(y_val,series)      

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=6, units=3, kernel_initializer="uniform")`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=2, kernel_initializer="uniform")`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  if __name__ == '__main__':


0.7487974683544305

In [0]:
dt=DecisionTreeClassifier()
dt.fit(x_train,y_train)
d_pred=dt.predict(x_val)
d_prob=dt.predict_proba(x_val)
accuracy_score(y_val,d_pred)
df_dt=pd.DataFrame()
df_dt['Survived_k']=d_pred
survivedt=pd.get_dummies(df_dt.Survived_k,prefix='Survivedk')
df['Survivedt_0']=survivedt['Survivedk_0']
df['Survivedt_1']=survivedt['Survivedk_1']
print("Accuracy score of DT",roc_auc_score(y_val,d_pred))

Accuracy score of DT 0.7118354430379746


In [0]:
from sklearn.naive_bayes import GaussianNB
gb=GaussianNB()
gb.fit(x_train,y_train)
g_pred=gb.predict(x_val)
g_prob=gb.predict_proba(x_val)
accuracy_score(y_val,g_pred)
df_gb=pd.DataFrame()
df_gb['Survived_k']=g_pred
survivedg=pd.get_dummies(df_gb.Survived_k,prefix='Survivedg')
df['Survivedg_0']=survivedg['Survivedg_0']
df['Survivedg_1']=survivedg['Survivedg_1']
print("Accuracy score of DT",roc_auc_score(y_val,g_pred))

Accuracy score of DT 0.7604430379746836


In [0]:
import xgboost as xgb
xgb=xgb.XGBClassifier()
xgb.fit(x_train,y_train)
x_pred=xgb.predict(x_val)
x_prob=xgb.predict_proba(x_val)
roc_auc_score(y_val,x_pred)

0.7534810126582278

In [0]:
df

,Survived_0,Survived_1,Surviveds_0,Surviveds_1,Survivedt_0,Survivedt_1,Survivedg_0,Survivedg_1
0,1,0,0,1,1,0,0,1
1,1,0,1,0,1,0,1,0
2,1,0,1,0,1,0,1,0
3,1,0,1,0,1,0,1,0
4,1,0,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...
174,0,1,0,1,1,0,0,1
175,0,1,0,1,0,1,0,1
176,1,0,1,0,1,0,1,0
177,0,1,0,1,0,1,0,1


**Implementing Random Forest for comparing the ensemble model with the Random Forest model**

In [0]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier()
forest.fit(x_train, y_train)
Y_pred = forest.predict(x_val)
print("Accuracy score of Random Forest",roc_auc_score(y_val,Y_pred))

Accuracy score of Random Forest 0.7334810126582278


In [0]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(x_train, y_train)
Y_pred2 = knn.predict(x_val)
print("Accuracy score of Kneighbors",roc_auc_score(y_val,Y_pred2))

Accuracy score of Kneighbors 0.7334810126582278


In [0]:
from sklearn.ensemble import AdaBoostClassifier
adb=AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=100)
adb.fit(x_train,y_train)
a_pred=adb.predict(x_val)
print("Auc score", roc_auc_score(y_val,a_pred))

Auc score 0.7604430379746836


**Same, comparing with the Kneareast neighbours algorithm**

In [0]:
df

,Survived_0,Survived_1,Surviveds_0,Surviveds_1,Survivedt_0,Survivedt_1,Survivedg_0,Survivedg_1
0,1,0,0,1,1,0,0,1
1,1,0,1,0,1,0,1,0
2,1,0,1,0,1,0,1,0
3,1,0,1,0,1,0,1,0
4,1,0,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...
174,0,1,0,1,1,0,0,1
175,0,1,0,1,0,1,0,1
176,1,0,1,0,1,0,1,0
177,0,1,0,1,0,1,0,1


**Generating normalized matrices Km and Kc using the method mentioned in paper 2.**

In [0]:
m=np.array(df)
s=np.zeros(shape=(m.shape[0],m.shape[0]))
for i in range(m.shape[0]):
  for j in range(m.shape[0]):
    if i!=j:
      sum1=0
      for k in range(2,m.shape[1]):
        sum1=sum1+(m[i][k]*m[j][k])
      s[i][j]=sum1
w=s
d=np.zeros(shape=w.shape[0])
for i in range(d.shape[0]):
  d[i]=np.sum(w[i],axis=0)
wnew=w/d.reshape(d.shape[0],1)
Km=wnew     

**Checking whether the matrices have been normalized or not i.e making them bistochastic **

In [0]:
print(np.sum(Km,axis=1))
print(np.sum(Km[0],axis=0))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
1.0


Similarly for Kc we generate a bistochastic matrix where m=the co-occurence matrix.

In [0]:
m=np.array(df)
s=np.zeros(shape=(m.shape[0],m.shape[0]))
for i in range(m.shape[0]):
  for j in range(m.shape[0]):
    if i!=j:
      sum1=0
      for k in range(2,m.shape[1]):
        sum1=sum1+(m[i][k]*m[j][k])
      s[i][j]=sum1
m=s   
s=np.zeros(shape=(m.shape[0],m.shape[0]))
for i in range(m.shape[0]):
  for j in range(m.shape[0]):
    if i!=j:
      sum1=0
      for k in range(2,m.shape[1]):
        sum1=sum1+(m[i][k]*m[j][k])
      s[i][j]=sum1   
d=np.zeros(shape=w.shape[0])
w=s
for i in range(d.shape[0]):
  d[i]=np.sum(w[i],axis=0)
wnew=w/d.reshape(d.shape[0],1)  
Kc=wnew
Kc

array([[0.        , 0.00552998, 0.00548787, ..., 0.00548787, 0.00550191,
        0.00550191],
       [0.00324558, 0.        , 0.00827869, ..., 0.00827869, 0.00034598,
        0.00034598],
       [0.00324897, 0.00835092, 0.        , ..., 0.00827614, 0.00034899,
        0.00034899],
       ...,
       [0.00324897, 0.00835092, 0.00827614, ..., 0.        , 0.00034899,
        0.00034899],
       [0.01137749, 0.00121902, 0.00121902, ..., 0.00121902, 0.        ,
        0.01384455],
       [0.01137749, 0.00121902, 0.00121902, ..., 0.00121902, 0.01384455,
        0.        ]])

**Same, checking for bi-stochastic behaviour**

In [0]:
  np.sum(Kc,axis=1)
  np.sum(Kc[0],axis=0)

1.0

**Generating Fg and Fo i'e the Group class matrix and Object Class matrix**

In [0]:
objclass_mat1=pd.DataFrame(y_prob,columns=['Survived','NotSurvived'])
objclass_mat2=pd.DataFrame(y_prob3,columns=['Survived','NotSurvived'])
objclass_mat3=pd.DataFrame(d_prob,columns=['Survived','NotSurvived'])
objclass_mat4=pd.DataFrame(g_prob,columns=['Survived','NotSurvived'])
df_t=pd.DataFrame(objclass_mat1)
df_t['Survived2']=objclass_mat2['Survived']
df_t['NotSurvived2']=objclass_mat2['NotSurvived']
df_t['Survived3']=objclass_mat3['Survived']
df_t['NotSurvived3']=objclass_mat3['NotSurvived']
df_t['Survived4']=objclass_mat4['Survived']
df_t['NotSurvived4']=objclass_mat4['NotSurvived']
G=8
C=2
m=G/2
n=df_t.shape[0]
dft=np.array(df_t)
Fo=np.zeros(shape=(n,C))
for i in range(n):
  sum1=0
  for j in range(0,G,2):
    sum1=sum1+dft[i][j]
  Fo[i][0]=sum1/m 
  sum2=0
  for j in range(1,G,2):
    sum2=sum2+dft[i][j]
  Fo[i][1]=sum2/m
gpclass_mat=pd.DataFrame(columns=['Survived','NotSurvived'],index=['class1surv','class1notsurv','class2surv','class2notsurv'])
prob1s=np.sum(objclass_mat1.Survived)/len(objclass_mat1)
prob1ns=np.sum(objclass_mat1.NotSurvived)/len(objclass_mat1)
prob2s=np.sum(objclass_mat2.Survived)/len(objclass_mat2)
prob2ns=np.sum(objclass_mat2.NotSurvived)/len(objclass_mat2)
list1=[]
list1.append(prob1s)
list1.append(prob1ns)
list1.append(prob2s)
list1.append(prob2ns)

list2=[]
list2.append(prob1ns)
list2.append(prob1s)
list2.append(prob2ns)
list2.append(prob2s)
gpclass_mat['Survived']=list1
gpclass_mat['NotSurvived']=list2
gpclass_mat
Fg=np.array(gpclass_mat)
Fo

array([[0.39013167, 0.60986833],
       [0.92294581, 0.07705419],
       [0.95534699, 0.04465301],
       [0.8020539 , 0.1979461 ],
       [0.36061374, 0.63938626],
       [0.90204205, 0.09795795],
       [0.73392061, 0.26607939],
       [0.90204205, 0.09795795],
       [0.14047208, 0.85952792],
       [0.80047982, 0.19952018],
       [0.89544323, 0.10455677],
       [0.73392061, 0.26607939],
       [0.90204205, 0.09795795],
       [0.90204205, 0.09795795],
       [0.04245952, 0.95754048],
       [0.91355798, 0.08644202],
       [0.89544323, 0.10455677],
       [0.21499363, 0.78500637],
       [0.20136344, 0.79863656],
       [0.87750882, 0.12249118],
       [0.91355798, 0.08644202],
       [0.87120663, 0.12879337],
       [0.27334598, 0.72665402],
       [0.90204205, 0.09795795],
       [0.87750882, 0.12249118],
       [0.39013167, 0.60986833],
       [0.90204205, 0.09795795],
       [0.90204205, 0.09795795],
       [0.90204205, 0.09795795],
       [0.0850197 , 0.9149803 ],
       [0.

In [0]:
Fo

array([[0.39013167, 0.60986833],
       [0.92294581, 0.07705419],
       [0.95534699, 0.04465301],
       [0.8020539 , 0.1979461 ],
       [0.36061374, 0.63938626],
       [0.90204205, 0.09795795],
       [0.73392061, 0.26607939],
       [0.90204205, 0.09795795],
       [0.14047208, 0.85952792],
       [0.80047982, 0.19952018],
       [0.89544323, 0.10455677],
       [0.73392061, 0.26607939],
       [0.90204205, 0.09795795],
       [0.90204205, 0.09795795],
       [0.04245952, 0.95754048],
       [0.91355798, 0.08644202],
       [0.89544323, 0.10455677],
       [0.21499363, 0.78500637],
       [0.20136344, 0.79863656],
       [0.87750882, 0.12249118],
       [0.91355798, 0.08644202],
       [0.87120663, 0.12879337],
       [0.27334598, 0.72665402],
       [0.90204205, 0.09795795],
       [0.87750882, 0.12249118],
       [0.39013167, 0.60986833],
       [0.90204205, 0.09795795],
       [0.90204205, 0.09795795],
       [0.90204205, 0.09795795],
       [0.0850197 , 0.9149803 ],
       [0.

**Similarly, generating  Yo and Yg**

In [0]:
n=4
C=survived.shape[1]
G=survived.shape[1]*n

Yo=np.zeros(shape=(df.shape[0],C))
cal_y=np.array(df)
for i in range(Yo.shape[0]):
  Yo[i][0]=(cal_y[i][0]+cal_y[i][2])/C
for i in range(Yo.shape[0]):
  Yo[i][1]=(cal_y[i][1]+cal_y[i][3])/C

In [0]:
G

8

In [0]:
m=np.array(df)
Yg=np.zeros(shape=(m.shape[1],2))
c=np.sum(m,axis=0)
for i in range(Yg.shape[0]):
  if i%2==0:
    Yg[i][0]=c[i]/m.shape[0]
    Yg[i][1]=1-Yg[i][0]
  else:
    Yg[i][1]=c[i]/m.shape[0]
    Yg[i][0]=1-Yg[i][1]  
Yg

array([[0.67039106, 0.32960894],
       [0.67039106, 0.32960894],
       [0.62569832, 0.37430168],
       [0.62569832, 0.37430168],
       [0.74301676, 0.25698324],
       [0.74301676, 0.25698324],
       [0.62011173, 0.37988827],
       [0.62011173, 0.37988827]])

Genearting 

Checking for the conditions specified in eq 6

In [0]:
np.linalg.norm((Fo[1]), ord=1)

1.0

In [0]:
Fo_cond=True
Fg_cond=True
for i in range(Fo.shape[0]):
  if np.linalg.norm((Fo[i]), ord=1)!=1:
    Fo_cond=False
for i in range(Fg.shape[0]):
  if np.linalg.norm((Fg[i]), ord=1)!=1:
    Fg_cond=False    
print(Fo_cond)
print(Fg_cond)

False
True


**Generating diagonal matrices Dm and Dc**

In [0]:
test_dm=np.zeros(shape=(Km.shape))
test_Km=Km
for i in range(test_Km.shape[0]):
  test_dm[i]=np.sum(test_Km[i],axis=0)
Dm=np.diag(np.diag(test_dm))
print("Shape of DM",Dm.shape)
test_dc=np.zeros(shape=(Kc.shape))
test_Kc=Kc
for i in range(test_Kc.shape[0]):
  test_dc[i]=np.sum(test_Kc[i],axis=0)
Dc=np.diag(np.diag(test_dc))
print("shape of Dc",Dc.shape)

Shape of DM (179, 179)
shape of Dc (179, 179)


In [0]:
print("KHM",Km.shape)
print("KHC",Kc.shape)
print("Fo",Fo.shape)
print("Fg",Fg.shape)
print("Yo",Yo.shape)
print("Yg",Yg.shape)
print("Dm",Dm.shape)
print("Dc",Dc.shape)

KHM (179, 179)
KHC (179, 179)
Fo (179, 2)
Fg (4, 2)
Yo (179, 2)
Yg (8, 2)
Dm (179, 179)
Dc (179, 179)


In [0]:
Yg=np.pad(Yg,pad_width=[(0, Km.shape[0]-G), (0, 0)])
Yg.shape

(179, 2)

In [0]:
print("KHM",Km.shape)
print("KHC",Kc.shape)
print("Fo",Fo.shape)
print("Fg",Fg.shape)
print("Yo",Yo.shape)
print("Yg",Yg.shape)
print("Dm",Dm.shape)
print("Dc",Dc.shape)

KHM (179, 179)
KHC (179, 179)
Fo (179, 2)
Fg (4, 2)
Yo (179, 2)
Yg (179, 2)
Dm (179, 179)
Dc (179, 179)


In [0]:
import sys
threshold=sys.float_info.epsilon

**Implementing algorithm 2**

In [0]:
alpha=0.40
beta=0.35
gamma=0.45
delta=0.5
Fo_t=np.zeros(shape=Fo.shape)
Fg_t=np.zeros(shape=Fg.shape)
Fot_old=np.zeros(shape=Fo.shape)
Fo_t=Fo
Fg_t=Fg
while LA.norm((Fo_t-Fot_old),'fro')>threshold:
  Fot_old=Fo_t
  Fg_t=np.matmul((np.linalg.inv(2*delta*np.ones(shape=Dm.shape)+(alpha*Dm))),((alpha*np.matmul(Km,Fo_t))+(2*delta*Yg)))
  Fo_t=np.matmul((np.linalg.inv((alpha*Dm)+(2*beta*Dc)-(beta*np.matmul(np.identity(Kc.shape[0]),Kc))-(beta*np.matmul(np.ones(shape=Kc.shape),Kc))+(2*gamma*np.identity(Kc.shape[0])))),((alpha*np.matmul(Km,Fg_t))+(2*gamma*Yo)))

In [0]:
from sklearn.metrics import accuracy_score
result=np.zeros(shape=Fo_t.shape[0])
for i in range(Fo_t.shape[0]):
  if Fo_t[i][0]<Fo_t[i][1]:
    result[i]=1
print("Accuracy value for Ensemble model",accuracy_score(y_val,result))
print("AUC for for Ensemble model",roc_auc_score(y_val,result))
print("Accuracy value for LR",accuracy_score(y_val,ypred))
print("AUC for for LR",roc_auc_score(y_val,ypred))
print("Accuracy value for bagging Random Forest",accuracy_score(y_val,Y_pred))
print("AUC for for bagging Random Forest",roc_auc_score(y_val,Y_pred))


Accuracy value for Ensemble model 0.776536312849162
AUC for for Ensemble model 0.7667721518987343
Accuracy value for LR 0.7653631284916201
AUC for for LR 0.7487974683544305
Accuracy value for bagging Random Forest 0.7541899441340782
AUC for for bagging Random Forest 0.7334810126582278


**We see that the ensemble model performs better than the Random forest, Kneigbours and Svm models and is at par with the Logistic regression model that gives us the best accuracy. **